#                                   Mask Mandate Research  

##  Morteza Maleki
May 28, 2021








###  





# Part 1: Housekeeping in R

In [1]:
rm(list = ls())

library(dplyr)
library(ggplot2)
library(readr)
library(tidyverse)
library(viridis)
library(ggmap)
library(sf)
library(data.table)
library(bit64)



setwd("C:/Users/momaleki/Desktop/Project")
getwd()


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v tibble  2.1.1     v purrr   0.3.2
v tidyr   0.8.3     v stringr 1.4.0
v tibble  2.1.1     v forcats 0.4.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning message:
"package 'viridis' was built under R version 3.6.3"Loading required package: viridisLite
Warning message:
"package 'viridisLite' was built under R version 3.6.3"Warning message:
"package 'ggmap' was buil

[1] "C:/Users/momaleki/Desktop/Project"

# 

# Part 2: Data Importation 


In [2]:

mask_use <- fread("Data/mask-use-by-county.csv")
COVID_closures <- fread("Data/US County Level Summaries.csv")
FIPS_master <- fread("Data/county_fips_master.csv")
Education_level <- fread("Data/Education level by Country 1970 - 2019.csv")
Election_decisions <- fread("Data/election decisions by fips country.csv")
Pop_estimates <- fread("Data/PopulationEstimates per country 2019.csv")
Poverty_estimate <- fread("Data/PovertyEstimates per country 2019.csv")
Presidential_elections <- fread("Data/presidential election results per fips county.csv")
Unemployment <- fread("Data/Unemployment per country 2000 -2019.csv")
FIPS_to_longlat <- fread("Data/2020_Gaz_counties_national.txt")
SCI <- fread(file = "Data/county_county_aug2020.tsv", sep = '\t', header = TRUE)
movement_range <- fread("Data/movement-range-2021-05-23.txt")
Population_density <- fread("Data/Population_density.csv")
Rural_percentage <- fread("Data/Rureal_Percentages_csv.csv") 
crime <- fread("Data/crime_data_w_population_and_crime_rate.csv")

# COVID Rate Data 
COVID_essential_workers <- fread("Data/COVID_Data/data/context_essential_workers_acs-2021-05-29.csv") 
COVID_testing_clinics <- fread("Data/COVID_Data/data/COVID_testing_clinics_PYTHON.csv") 
COVID_hospital_locations <- fread("Data/COVID_Data/data/COVID_Hospitals_PYTHON.csv") 
COVID_confirmed_cases <- fread("Data/COVID_Data/data/covid_confirmed_cdc-2021-05-29.csv") 
COVID_death_cases <- fread("Data/COVID_Data/data/covid_deaths_cdc-2021-05-29.csv")
COVID_testing_counts <- fread("Data/COVID_Data/data/covid_testing_cdc-2021-05-29.csv") 
COVID_testing_capacity <- fread("Data/COVID_Data/data/covid_tcap_cdc-2021-05-29.csv") 
COVID_testing_positivity <- fread("Data/COVID_Data/data/covid_wk_pos_cdc-2021-05-29.csv") 
Vaccine <- fread("Data/COVID_Data/data/COVID-19_Vaccinations_in_the_United_States_County.csv")

#############
FIPS_master$county_name <- paste(FIPS_master$county_name, FIPS_master$state_abbr, sep = ", ")
Crime <- merge(FIPS_master, crime, on = "county_name")
Crime <- Crime[, -c(1, 3:13, 15:36)]
colnames(Crime) <- c('FIPS', 'Crime_per_100K')
head(Crime)

FIPS,Crime_per_100K
45001,511.85668
22001,163.97659
51001,190.05672
16001,206.50044
19001,66.91649
21001,96.09225


# Part 3: Data Preprocessing

In [3]:
# Making sure we have the same FIPS code in all dataset with type double 

# mask_use
colnames(mask_use) <- paste("Mask_Usage", colnames(mask_use), sep = "_")
mask_use$FIPS <- as.double(mask_use$Mask_Usage_COUNTYFP)
mask_use$Mask_Usage_COUNTYFP <- NULL
mask_use$Mask_Usage_Combined <- mask_use[, c(1)] * 0 + mask_use[, c(2)] * 2 +
                                    mask_use[, c(3)] * 3 + mask_use[, c(4)] * 4 +
                                            mask_use[, c(5)] * 5






# FIPS_master
FIPS_master$FIPS <- as.double(FIPS_master$fips)
FIPS_master$fips <- NULL
FIPS_master <- FIPS_master[, -c(4:12)]




# Education Level
colnames(Education_level) <- paste("Education_Levels", colnames(Education_level), sep = "_")
Education_level$FIPS <- as.double(Education_level$`Education_Levels_FIPS Code`)
Education_level$`Education_Levels_FIPS Code` <- NULL
Education_level <- Education_level[, -c(0:38)]
Education_level <- subset(Education_level, floor(log10(FIPS))  == 4)
                    
colnames(Education_level) <- c('Count_Less_HighSchool', 'Count_HighSchool', 'Count_Less_College', 
                              'Count_Higher_College', 'Percent_Less_HighSchool', 'Percent_HighSchool', 
                             'Percent_Less_College', 'Percent_Higher_College', 'FIPS')

Education_level$Percent_Less_HighSchool <- Education_level$Percent_Less_HighSchool/100
Education_level$Percent_HighSchool <- Education_level$Percent_HighSchool/100
Education_level$Percent_Less_College <- Education_level$Percent_Less_College/100
Education_level$Percent_Higher_College <- Education_level$Percent_Higher_College/100





# COVID Closures
colnames(COVID_closures) <- paste("COVID_Closures", colnames(COVID_closures), sep = "_")
COVID_closures$FIPS <- as.double(COVID_closures$COVID_Closures_FIPS)
COVID_closures$COVID_Closures_FIPS <- NULL



#  Election Decisions 
colnames(Election_decisions) <- paste("Election", colnames(Election_decisions), sep = "_")
Election_decisions$FIPS <- as.double(Election_decisions$Election_id)
Election_decisions <- Election_decisions[, -c(2, 3, 4, 5, 11, 12 )]
Election_decisions$Election_Trump <- ifelse(Election_decisions$Election_lname == "Trump", 1, 0)
Election_decisions$Election_Biden <- ifelse(Election_decisions$Election_lname == "Biden", 1, 0)
Election_decisions$Trump_Votes <- Election_decisions$Election_votes * Election_decisions$Election_Trump
Election_decisions$Biden_Votes <- Election_decisions$Election_votes * Election_decisions$Election_Biden

Election_decisions_1 <- Election_decisions %>% 
                            filter(Election_race == "President") %>%
                            group_by(FIPS) %>%
                                summarize(Winner_Votes = max(Election_votes), 
                                         Trump_Votes = max(Trump_Votes), 
                                         Biden_Votes = max(Biden_Votes))
Election_decisions_1$Trump_Win <- ifelse(Election_decisions_1$Trump_Votes > Election_decisions_1$Biden_Votes, 
                                  1, 0)
Election_decisions_1$Biden_Win <- ifelse(Election_decisions_1$Trump_Votes < Election_decisions_1$Biden_Votes, 
                                  1, 0)
Election_decisions_1 <- subset(Election_decisions_1, floor(log10(FIPS))  == 4)
Election_decisions_1 <- Election_decisions_1[-c(2:4)]
Election_decisions <- Election_decisions_1





# FIPS_to_longlat
FIPS_to_longlat$FIPS <- FIPS_to_longlat$GEOID
FIPS_to_longlat$GEOID <- NULL
FIPS_to_longlat$LAT <- FIPS_to_longlat$INTPTLAT
FIPS_to_longlat$LON <- FIPS_to_longlat$INTPTLONG
FIPS_to_longlat <- FIPS_to_longlat[, -c(0:9)]




#  Unemployment 
colnames(Unemployment) <- paste("Unemployment_2019", colnames(Unemployment), sep = "_")
Unemployment$FIPS <- as.double(Unemployment$Unemployment_2019_fips_txt)
keep <- c("Unemployment_rate_2019")
Unemployment <- subset(Unemployment, Unemployment_2019_Attribute %in% keep)
Unemployment <- Unemployment[, -c(1:4)]



#  Povery Estimate
colnames(Poverty_estimate) <- paste("Poverty_Estimates", colnames(Poverty_estimate), sep = "_")
Poverty_estimate$FIPS <- as.double(Poverty_estimate$Poverty_Estimates_FIPStxt)
keep <- c('POVALL_2019')
Poverty_estimate <- subset(Poverty_estimate, Poverty_Estimates_Attribute %in% keep)
Poverty_estimate <- Poverty_estimate[, -c(1:4)]


# Movement Range
movement_range$date <- movement_range$ds
movement_range$ds <- NULL
movement_range$country_code <- movement_range$country
movement_range$country <- NULL
movement_range$polygon_source <- NULL
movement_range$polygon_name <- NULL
movement_range$change_in_movement <- movement_range$all_day_bing_tiles_visited_relative_change
movement_range$all_day_bing_tiles_visited_relative_change <- NULL
movement_range$proportion_intouch_users <- movement_range$all_day_ratio_single_tile_users
movement_range$all_day_ratio_single_tile_users <- NULL
movement_range$baseline_calc_time <- movement_range$baseline_name
movement_range$baseline_name <- NULL
movement_range$baseline_calc_how <- movement_range$baseline_type
movement_range$baseline_type <- NULL
movement_range = filter(movement_range, movement_range$country_code == "USA")
movement_range$date <- as.Date(movement_range$date, format = "%Y-%m-%d")
movement_range = subset(movement_range, date > "2020-07-1" & date < "2020-07-15")
colnames(movement_range) <- paste("Movement_Range", colnames(movement_range), sep = "_")
movement_range$FIPS <- as.double(movement_range$Movement_Range_polygon_id)
movement_range$Movement_Range_polygon_id <- NULL
movement_range <- movement_range[, -c(2, 5, 6)]

movement_range <- movement_range %>% 
                    group_by(FIPS) %>%
                            summarize(Change_in_Movement = mean(Movement_Range_change_in_movement),
                                      Intouch_users_Proportion = mean(Movement_Range_proportion_intouch_users))





# SCI
colnames(SCI) <- paste("SCI", colnames(SCI), sep = "_")
SCI$FIPS <- as.double(SCI$SCI_user_loc)
SCI$SCI_user_loc <- NULL
SCI$second_FIPS <- as.double(SCI$SCI_fr_loc)
SCI$SCI_fr_loc <- NULL
SCI$Multiply <- SCI$FIPS * SCI$second_FIPS * SCI$SCI_scaled_sci
SCI_Unique <- distinct(SCI, Multiply, .keep_all = TRUE)
SCI <- SCI_Unique
SCI$Multiply <- NULL





# Population Density
colnames(Population_density) <- paste("Pop_Density", colnames(Population_density), sep = "_")
Population_density$FIPS <- as.double(Population_density$Pop_Density_GEOID)
Population_density$Population_Density <- as.double(Population_density$Pop_Density_B01001_calc_PopDensity)
Population_density <- Population_density[, -c(1:23)]




# Rural Population 
colnames(Rural_percentage) <- paste("Rural_Percent", colnames(Rural_percentage), sep = "_")
Rural_percentage$FIPS <- Rural_percentage[, c(1)]
Rural_percentage <- Rural_percentage[, -c(1:4)]
Rural_percentage_1 <- Rural_percentage
colnames(Rural_percentage_1) <- c('Total_Population', 'Urban_Population', 'Rural_Population', 'Percent_Rural', 'FIPS')
Rural_percentage_1$Percent_Rural <- Rural_percentage_1$Percent_Rural / 100
Rural_percentage <- Rural_percentage_1





##### COVID Data 

# Vaccination 
Vaccine <- Vaccine %>%
                  group_by(FIPS) %>%
                            summarize(Vaccine_completeness_pct = mean(Series_Complete_Pop_Pct))

Vaccine$FIPS <- as.double(Vaccine$FIPS)
Vaccine$Vaccine_completeness_pct <- Vaccine$Vaccine_completeness_pct / 100

# COVID Essential Workers: percent of essential workers
colnames(COVID_essential_workers) <- paste("COVID_Essential_Workers", colnames(COVID_essential_workers), sep = "_")
COVID_essential_workers$FIPS <- COVID_essential_workers$COVID_Essential_Workers_fips
COVID_essential_workers$COVID_Essential_Workers_fips <- NULL

# COVID Testing Clinics: 
colnames(COVID_testing_clinics) <- paste("COVID_Clinics", colnames(COVID_testing_clinics), sep = "_")
COVID_testing_clinics$COVID_Clinic_Testing <- ifelse(COVID_testing_clinics$'COVID_Clinics_testing_status' == "Yes", 1, 0)
COVID_testing_clinics <- COVID_testing_clinics[, -c(1:7, 9:10)]
COVID_testing_clinics$FIPS <- COVID_testing_clinics$COVID_Clinics_FIPS
COVID_testing_clinics <- COVID_testing_clinics[, -c(1:2)]


# COVID Hospitcal Locations:
colnames(COVID_hospital_locations) <- paste("COVID_Hospital", colnames(COVID_hospital_locations), sep = "_")
COVID_hospital_locations <- COVID_hospital_locations[, -c(1:11 ,17:25)]
COVID_hospital_locations[is.na(COVID_hospital_locations)] <- 0
COVID_hospital_locations$COVID_Hospital_All_Beds <- rowSums(COVID_hospital_locations[, c(1,  3)])
COVID_hospital_locations$COVID_Hospital_Occupancy_Rate <- rowMeans(COVID_hospital_locations[, c(4, 5)])
COVID_hospital_locations$FIPS <- COVID_hospital_locations$COVID_Hospital_FIPS
COVID_hospital_locations <- COVID_hospital_locations[, -c(1:6)]




# COVID Confirmed Cases:
colnames(COVID_confirmed_cases) <- paste("COVID_Confirmed", colnames(COVID_confirmed_cases), sep = "_")
COVID_confirmed_cases$FIPS <- as.double(COVID_confirmed_cases$COVID_Confirmed_fips_code)
COVID_confirmed_cases <- COVID_confirmed_cases[, -c(2:163, 177:493)]
COVID_confirmed_cases$COVID_Average_Confirmed <- rowMeans(COVID_confirmed_cases[, c(2:14)])
COVID_confirmed_cases$FIPS <- COVID_confirmed_cases$COVID_Confirmed_fips_code
COVID_confirmed_cases <- COVID_confirmed_cases[, -c(1:14)]

# COVID Death Cases:
colnames(COVID_death_cases) <- paste("COVID_Death", colnames(COVID_death_cases), sep = "_")
COVID_death_cases<- COVID_death_cases[, -c(2:163, 177:492)]
COVID_death_cases$COVID_Average_Death <- rowMeans(COVID_death_cases[, c(2:14)])
COVID_death_cases$FIPS <- COVID_death_cases$COVID_Death_fips_code
COVID_death_cases <- COVID_death_cases[, -c(1:14)]

# COVID Testing Counts: 
colnames(COVID_testing_counts) <- paste("COVID_TestCounts", colnames(COVID_testing_counts), sep = "_")
COVID_testing_counts <- COVID_testing_counts[, -c(2:163, 177:492)]
COVID_testing_counts$COVID_Average_Testing <- rowMeans(COVID_testing_counts[, c(2:14)])
COVID_testing_counts$FIPS <- COVID_testing_counts$COVID_TestCounts_fips_code
COVID_testing_counts <- COVID_testing_counts[, -c(1:14)]

# COVID Testing Capacity:
colnames(COVID_testing_capacity) <- paste("COVID_TestCap", colnames(COVID_testing_capacity), sep = "_")
COVID_testing_capacity <- COVID_testing_capacity[, -c(2:163, 177:492)]
COVID_testing_capacity$COVID_Average_TestCapacity <- rowMeans(COVID_testing_capacity[, c(2:14)])
COVID_testing_capacity$FIPS <- COVID_testing_capacity$COVID_TestCap_fips_code
COVID_testing_capacity <- COVID_testing_capacity[, -c(1:14)]

Warning message in eval(expr, envir, enclos):
"NAs introduced by coercion"

## Part 3: Data Processing ----  Looking at the Data

In [10]:
head(mask_use)

Mask_Usage_NEVER,Mask_Usage_RARELY,Mask_Usage_SOMETIMES,Mask_Usage_FREQUENTLY,Mask_Usage_ALWAYS,FIPS,Mask_Usage_Combined
0.053,0.074,0.134,0.295,0.444,1001,3.950
0.083,0.059,0.098,0.323,0.436,1003,3.884
0.067,0.121,0.120,0.201,0.491,1005,3.861
0.020,0.034,0.096,0.278,0.572,1007,4.328
0.053,0.114,0.180,0.194,0.459,1009,3.839
0.031,0.040,0.144,0.286,0.500,1011,4.156


In [11]:
head(FIPS_master[FIPS_master$state_name == "Delaware"])

county_name,state_abbr,state_name,FIPS
"Kent County, DE",DE,Delaware,10001
"New Castle County, DE",DE,Delaware,10003
"Sussex County, DE",DE,Delaware,10005


In [12]:
head(Education_level)

Count_Less_HighSchool,Count_HighSchool,Count_Less_College,Count_Higher_College,Percent_Less_HighSchool,Percent_HighSchool,Percent_Less_College,Percent_Higher_College,FIPS
"66,816","209,449","178,917","214,138",0.100,0.313,0.267,0.320,10000
"14,834","38,843","36,347","28,030",0.126,0.329,0.308,0.237,10001
"32,032","118,233","94,434","138,627",0.084,0.308,0.246,0.362,10003
"19,950","52,373","48,136","47,481",0.119,0.312,0.287,0.283,10005
"44,850","83,185","76,822","289,259",0.091,0.168,0.155,0.585,11000
"44,850","83,185","76,822","289,259",0.091,0.168,0.155,0.585,11001


In [13]:
head(Election_decisions, 10)

FIPS,Trump_Win,Biden_Win
10001,0,1
10003,0,1
10005,1,0
12001,0,1
12003,1,0
12005,1,0
12007,1,0
12009,1,0
12011,0,1
12013,1,0


In [14]:
head(FIPS_to_longlat)

FIPS,LAT,LON
1001,32.53224,-86.64644
1003,30.65922,-87.74607
1005,31.87025,-85.40510
1007,33.01589,-87.12715
1009,33.97736,-86.56644
1011,32.10176,-85.71726


In [15]:
head(Unemployment)

Unemployment_2019_Value,FIPS
3.669409,0
3.000000,1000
2.700000,1001
2.700000,1003
3.800000,1005
3.100000,1007


In [16]:
head(movement_range, 5)

FIPS,Change_in_Movement,Intouch_users_Proportion
1001,-0.05196923,0.1953131
1003,0.04598385,0.1708885
1005,-0.06379846,0.1729154
1007,-0.05512308,0.2020915
1009,-0.05228846,0.1856385


In [17]:
head(SCI)

SCI_scaled_sci,FIPS,second_FIPS
13971142,1001,1001
145133,1001,1003
200193,1001,1005
371688,1001,1007
95859,1001,1009
570502,1001,1011


In [18]:
head(Population_density)

FIPS,Population_Density
1001,35.85342
1003,50.54150
1005,11.24798
1007,13.97311
1009,34.51582
1011,6.41762


In [19]:
head(Rural_percentage)

Total_Population,Urban_Population,Rural_Population,Percent_Rural,FIPS
"54,571","31,650","22,921",0.420,1001
"182,265","105,205","77,060",0.423,1003
"27,457","8,844","18,613",0.678,1005
"22,915","7,252","15,663",0.684,1007
"57,322","5,760","51,562",0.900,1009
"10,914","5,307","5,607",0.514,1011


In [20]:
head(COVID_closures)

COVID_Closures_V2,COVID_Closures_V3,COVID_Closures_stay at home,COVID_Closures_>50 gatherings,COVID_Closures_>500 gatherings,COVID_Closures_public schools,COVID_Closures_restaurant dine-in,COVID_Closures_entertainment/gym,COVID_Closures_Federal guidelines,COVID_Closures_foreign travel ban,FIPS
alabama,AL,4-Apr,20-Mar,13-Mar,16-Mar,19-Mar,28-Mar,16-Mar,11-Mar,1000
autauga county,AL,4-Apr,20-Mar,13-Mar,16-Mar,19-Mar,28-Mar,16-Mar,11-Mar,1001
baldwin county,AL,4-Apr,20-Mar,13-Mar,16-Mar,19-Mar,28-Mar,16-Mar,11-Mar,1003
barbour county,AL,4-Apr,20-Mar,13-Mar,16-Mar,19-Mar,28-Mar,16-Mar,11-Mar,1005
bibb county,AL,4-Apr,20-Mar,13-Mar,16-Mar,19-Mar,28-Mar,16-Mar,11-Mar,1007
blount county,AL,4-Apr,20-Mar,13-Mar,16-Mar,19-Mar,28-Mar,16-Mar,11-Mar,1009


In [21]:
head(COVID_confirmed_cases)  # 7-day rolling average of new confirmed cases of Covid-19

COVID_Average_Confirmed,FIPS
16.830769,1001
93.984615,1003
10.300000,1005
5.592308,1007
15.792308,1009
2.507692,1011


In [4]:
head(Poverty_estimate)

Poverty_Estimates_Value,FIPS
39490096,0
747478,1000
6723,1001
22360,1003
5909,1005
4101,1007


In [22]:
head(COVID_death_cases) # 7-day rolling average of new deaths attributed to Covid-19.

COVID_Average_Death,FIPS
0.253846154,1001
0.123076923,1003
0.061538462,1005
0.007692308,1007
0.000000000,1009
0.053846154,1011


In [23]:
head(COVID_essential_workers)

COVID_Essential_Workers_pct_essential,FIPS
0.6130137,17051
0.5316272,17107
0.5724096,17165
0.3737092,17097
0.5440028,17127
0.6343259,17017


In [24]:
head(COVID_hospital_locations)

COVID_Hospital_All_Beds,COVID_Hospital_Occupancy_Rate,FIPS
0,0.000,18097
20,0.000,25009
0,0.000,17119
212,0.610,72099
742,0.505,34025
310,0.000,36087


In [25]:
head(COVID_testing_capacity) # 7-day rolling average of tests completed per 100k population in the county

COVID_Average_TestCapacity,FIPS
313.7231,1001
260.4000,1003
165.5154,1005
162.8385,1007
173.4000,1009
252.4462,1011


In [26]:
head(COVID_testing_clinics)

COVID_Clinic_Testing,FIPS
1,66010
1,66010
1,69120
1,69110
1,60050
1,60050


In [27]:
head(COVID_testing_counts) # 7-day rolling average of total tests completed

COVID_Average_Testing,FIPS
173.74615,1001
554.18462,1003
41.97692,1005
36.63846,1007
100.02308,1009
25.87692,1011


In [224]:
head(Vaccine)

FIPS,Vaccine_completeness_pct
1001,0.08196067
1003,0.11169101
1005,0.08088764
1007,0.07486517
1009,0.06486517
1011,0.12137079


In [225]:
head(Crime)

FIPS,Crime_per_100K
45001,511.85668
22001,163.97659
51001,190.05672
16001,206.50044
19001,66.91649
21001,96.09225


In [201]:
## Union in Python
write.csv(SCI, 'Data/ToMerge/SCI.csv', fileEncoding="UTF-8")
write.csv(mask_use, 'Data/ToMerge/mask_use.csv', fileEncoding="UTF-8")
write.csv(FIPS_master,'Data/ToMerge/FIPS_master.csv', fileEncoding="UTF-8")
write.csv(Education_level, 'Data/ToMerge/Education_level.csv', fileEncoding="UTF-8")
write.csv(Election_decisions, 'Data/ToMerge/Election_decisions.csv', fileEncoding="UTF-8")
write.csv(FIPS_to_longlat, 'Data/ToMerge/FIPS_to_longlat.csv', fileEncoding="UTF-8")
write.csv(Unemployment, 'Data/ToMerge/Unemployment.csv', fileEncoding="UTF-8")
write.csv(movement_range, 'Data/ToMerge/movement_range.csv', fileEncoding="UTF-8")
write.csv(Poverty_estimate, 'Data/ToMerge/Poverty_estimate.csv', fileEncoding="UTF-8")
write.csv(Population_density, 'Data/ToMerge/Population_density.csv', fileEncoding="UTF-8")
write.csv(Rural_percentage, 'Data/ToMerge/Rural_percentage.csv', fileEncoding="UTF-8")
write.csv(COVID_essential_workers, 'Data/ToMerge/COVID_essential_workers.csv', fileEncoding="UTF-8")
write.csv(COVID_testing_clinics , 'Data/ToMerge/COVID_testing_clinics.csv', fileEncoding="UTF-8")
write.csv(COVID_hospital_locations, 'Data/ToMerge/COVID_hospital_locations.csv',fileEncoding="UTF-8")
write.csv(COVID_confirmed_cases, 'Data/ToMerge/COVID_confirmed_cases.csv', fileEncoding="UTF-8")
write.csv(COVID_death_cases, 'Data/ToMerge/COVID_death_cases.csv', fileEncoding="UTF-8")
write.csv(COVID_testing_counts, 'Data/ToMerge/COVID_testing_counts.csv', fileEncoding="UTF-8")
write.csv(COVID_testing_capacity, 'Data/ToMerge/COVID_testing_capacity.csv', fileEncoding="UTF-8")
write.csv(Crime, 'Data/ToMerge/Crime.csv', fileEncoding="UTF-8")
write.csv(Vaccine, 'Data/ToMerge/Vaccine.csv', fileEncoding="UTF-8")